In [62]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [64]:
defense = pd.read_csv("../match_logs/Big5@22-23@defense.csv")
gca = pd.read_csv("../match_logs/Big5@22-23@gca.csv")
keeper = pd.read_csv("../match_logs/Big5@22-23@keeper.csv")
misc = pd.read_csv("../match_logs/Big5@22-23@misc.csv")
passing = pd.read_csv("../match_logs/Big5@22-23@passing.csv")
passing_types = pd.read_csv("../match_logs/Big5@22-23@passing_types.csv")
possession = pd.read_csv("../match_logs/Big5@22-23@possession.csv")
shooting = pd.read_csv("../match_logs/Big5@22-23@shooting.csv")

In [65]:
team_ratings = pd.read_csv("../players_db/fm23/team_ratings.csv")

In [66]:
team_ratings_cols = team_ratings.columns

In [67]:
tpr_cols = []
except_cols = ["Club","Based","Division","Club_id","League_id","fbref_name"]
for col in team_ratings_cols:
    if col not in except_cols:
        for xy in ["x","y"]:
            xy_col = f"{col}_{xy}"
            tpr_cols.append(xy_col)

<br><br><br><br><br><br><br><br>
<h1 style="color:orange;background:blue;">@.  Logs to Regression</h1>

In [68]:
CURRENT_DATAFRAME_NAME = "shooting"

In [69]:
CURRENT_DATAFRAME = locals()[CURRENT_DATAFRAME_NAME]
CURRENT_DATAFRAME["Unnamed: 0"] = "itsanobject"

In [70]:
# CURRENT_DATAFRAME["Unique_Matchday_id"] = ((CURRENT_DATAFRAME.Club).apply(hash)+(CURRENT_DATAFRAME.Opponent).apply(hash)).apply(str)
# unique_indexes = CURRENT_DATAFRAME["Unique_Matchday_id"].drop_duplicates().index
# CURRENT_DATAFRAME = CURRENT_DATAFRAME.loc[unique_indexes]

In [71]:
DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.columns)
NUMERIC_DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.select_dtypes(exclude="object").columns)

In [72]:
CURRENT_DATAFRAME[NUMERIC_DATAFRAME_COLUMNS].describe().loc[['count','min', 'mean', 'max']].T[["mean"]].apply(round)

,mean
GF,1.0
GA,1.0
Gls,1.0
Sh,12.0
SoT,4.0
SoT%,34.0
G/Sh,0.0
G/SoT,0.0
Dist,18.0
FK,0.0


In [73]:
merged_df_1 = pd.merge(CURRENT_DATAFRAME, team_ratings, on="Club", how="inner")  
merged_df = pd.merge(merged_df_1, team_ratings, left_on="Opponent", right_on="fbref_name", how="inner")  

In [74]:
merged_df["Versus_Title"] = merged_df.Club_x + " plays vs " + \
                             merged_df.Club_y + " as " + merged_df.Venue + " team"

In [75]:
MERGED_NUMERIC_DATAFRAME_COLUMNS = list(merged_df.select_dtypes(exclude="object").columns)

In [76]:
reg_df = merged_df[['Club_x','Club_y',"Versus_Title","Date"] + MERGED_NUMERIC_DATAFRAME_COLUMNS] 
reg_df.iloc[0]

Club_x                                                Manchester City
Club_y                                                West Ham United
Versus_Title        Manchester City plays vs West Ham United as Aw...
Date                                                       2022-08-07
GF                                                                2.0
                                          ...                        
Marking_y                                                          80
Passing_y                                                          67
Penalty Taking_y                                                   68
Tackling_y                                                         84
Technique_y                                                        75
Name: 0, Length: 101, dtype: object

<br><br><br><br><br><br><br><br>
<h2 style="color:white;background:red;">  Regression to get Feature Importance</h2>

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [78]:
formula_df = pd.DataFrame(index=tpr_cols)
np.random.seed(0)
df = reg_df.dropna(axis=0)

for attribute in NUMERIC_DATAFRAME_COLUMNS:
    X = df[tpr_cols]
    Y = df[[attribute]]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    model = LinearRegression()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    coef_column_name = f"{attribute}_coef"
    intercept_column_name = f"{attribute}_intercept"
    formula_df[coef_column_name] = model.coef_.flatten()
    formula_df[intercept_column_name] = model.intercept_[0]

    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    # print(f'{attribute} --- Mean Squared Error: {mse}')
    # print(f'{attribute} --- R^2 Score: {r2}')
    # print("\n")

In [79]:
with pd.ExcelWriter(f"{CURRENT_DATAFRAME_NAME}_Standardized_coefficients.xlsx") as writer:
    formula_df[formula_df.index.str.contains("_x")].to_excel(writer, index=True, sheet_name="x")
    formula_df[formula_df.index.str.contains("_y")].to_excel(writer, index=True, sheet_name="y")

<br><br><br><br>

<br><br><br><br><br><br><br><br>
<h2 style="color:#33FF49;background:blue;">  Regression for One Atrribute</h2>

In [80]:
attribute = "Sh"

In [81]:
tpr_x_dict = {
    "tpr_x": None,
    "Acceleration_x": None,
    "Technique_x": None,
    "Composure_x": None,
    "Finishing_x": None,
    "Agility_x": None,
    "Corners_x": None,
    "Long Shots_x": None,
    "Teamwork_x": None,
    "Work Rate_x": None,
    "Off the Ball_x": None,
    "Heading_x": None,
    "First Touch_x": None,
    "Crossing_x": None
}

In [82]:
tpr_y_dict = {
    "tpr_y": None,
    "Acceleration_y": None,
    "Agility_y": None,
    "Off the Ball_y": None,
    "Work Rate_y": None,
    "Teamwork_y": None,
    "Decisions_y": None,
    "Balance_y": None,
    "Positioning_y": None,
    "Marking_y": None
}

In [83]:
row_index2 = 3221 # for Bayern
row_index = 3758 # for Bochum

x_cols = tpr_x_dict.keys()
y_cols = tpr_y_dict.keys()

tpr_x_custom_data = reg_df[x_cols].iloc[row_index].to_dict()
tpr_y_custom_data = reg_df[y_cols].iloc[row_index].to_dict()

custom_data = {**tpr_x_custom_data, **tpr_y_custom_data}
custom_cols = custom_data.keys()

In [84]:
special_formula_df = pd.DataFrame(index=custom_cols)

df_to_custom = df.copy()
X = df_to_custom[custom_cols]
Y = df_to_custom[[attribute]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

coef_column_name = f"{attribute}_coef"
intercept_column_name = f"{attribute}_intercept"
special_formula_df[coef_column_name] = model.coef_.flatten()
special_formula_df[intercept_column_name] = model.intercept_[0]

mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

In [85]:
special_formula_df

,Sh_coef,Sh_intercept
tpr_x,0.062547,6.969175
Acceleration_x,0.076267,6.969175
Technique_x,0.039200,6.969175
Composure_x,0.040315,6.969175
Finishing_x,0.050389,6.969175
Agility_x,0.046719,6.969175
Corners_x,0.024361,6.969175
Long Shots_x,0.006254,6.969175
Teamwork_x,-0.007192,6.969175
Work Rate_x,-0.011190,6.969175


In [86]:
# query2 = reg_df.query("Versus_Title == 'VfL Bochum plays vs FC Bayern as Home team'")
# query = reg_df.query("Versus_Title == 'FC Bayern plays vs VfL Bochum as Home team'")
# query.iloc[0].Date
# query.iloc[0].name

In [87]:
custom_tpr_df = pd.DataFrame(custom_data, index=["tpr_value"]).T

In [88]:
custom_tpr_df[f"{attribute}_coef"] = special_formula_df[f"{attribute}_coef"][custom_cols].tolist()
custom_tpr_df[f"{attribute}_intercept"] = special_formula_df[f"{attribute}_intercept"][custom_cols].tolist()
custom_tpr_df["coef_result"] = custom_tpr_df[f"{attribute}_coef"] * custom_tpr_df["tpr_value"]
custom_tpr_df["formula_result"] = custom_tpr_df["coef_result"].sum() + custom_tpr_df[f"{attribute}_intercept"].iloc[0]
custom_tpr_df["formula_result"] = round(custom_tpr_df["formula_result"].iloc[0])

In [89]:
custom_tpr_df

,tpr_value,Sh_coef,Sh_intercept,coef_result,formula_result
tpr_x,64,0.062547,6.969175,4.003038,9
Acceleration_x,75,0.076267,6.969175,5.720030,9
Technique_x,60,0.039200,6.969175,2.351974,9
Composure_x,64,0.040315,6.969175,2.580157,9
Finishing_x,65,0.050389,6.969175,3.275314,9
Agility_x,66,0.046719,6.969175,3.083486,9
Corners_x,65,0.024361,6.969175,1.583454,9
Long Shots_x,59,0.006254,6.969175,0.368985,9
Teamwork_x,72,-0.007192,6.969175,-0.517792,9
Work Rate_x,71,-0.011190,6.969175,-0.794515,9


<br><br><br><br><br><br><br><br>
<h2 style="color:blue;background:aqua;">  Error Rates</h2>